# Spatial Querying of GEDI Version 2 Data in Python

The Global Ecosystem Dynamics Investigation ([GEDI](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/gedi-overview/)) mission aims to characterize ecosystem structure and dynamics to enable radically improved quantification and understanding of the Earth's carbon cycle and biodiversity. The Land Processes Distributed Active Archive Center (LP DAAC) distributes the GEDI Level 1 and Level 2 Version 1 and Version 2 products. The LP DAAC created the GEDI Finder _Web Service_ to allow users to perform spatial queries of GEDI _Version 1_ L1-L2 full-orbit granules. One of the updates for GEDI _Version 2_ included additional spatial metadata that allows users to perform spatial queries via a graphical user interface (GUI) using NASA's [Earthdata Search](https://search.earthdata.nasa.gov/search) or programmatically using NASA's [Common Metadata Repository](https://cmr.earthdata.nasa.gov/search) (CMR). Another update is that each GEDI V1 full-orbit granule has been divided into 4 sub-orbit granules in V2.   

### The objective of this tutorial is to demonstrate how current GEDI Finder users can update their workflow for GEDI Version 2 (V2) data using NASA's CMR to perform spatial [bounding box] queries for GEDI V2 L1B, L2A, and L2B data, and how to reformat the CMR response into a list of links that will allow users to download the intersecting GEDI V2 sub-orbit granules directly from the LP DAAC Data Pool. 

---
## Use Case Example:  
This tutorial was developed using an example use case for a current GEDI Finder user who has been using the GEDI Finder web service in Python to find intersecting GEDI L2A Version 1 full-orbit granules over the Amazon Rainforest. The user is now looking to use the same workflow to find intersecting GEDI L2A V2 sub-orbit granules. 

This tutorial will show how to use Python to perform a spatial query for GEDI V2 data using NASA's CMR, how to reformat the CMR response into a list of links pointing to the intersecting sub-orbit granules on the LP DAAC Data Pool, and how to export the list of links as a text file. 
***    
## Applicable Data Products: 
### This tutorial can be used to perform spatial queries on the following products:
- **GEDI L1B Geolocated Waveform Data Global Footprint Level - [GEDI01_B.002](https://doi.org/10.5067/GEDI/GEDI01_B.002)**
- **GEDI L2A Elevation and Height Metrics Data Global Footprint Level - [GEDI02_A.002](https://doi.org/10.5067/GEDI/GEDI02_A.002)**
- **GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level - [GEDI02_B.002](https://doi.org/10.5067/GEDI/GEDI02_B.002)**

***  
# Topics Covered:
1. [**Import Packages**](#importpackages)  
2. [**Define Function to Query CMR**](#definefunction)      
3. [**Execute GEDI_Finder Function**](#executefunction)      
4. [**Export Results**](#exportresults)        
***
# Before Starting this Tutorial:

  
***
## Source Code used to Generate this Tutorial:
#### The repository containing the files is located at: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-finder-tutorial-python/browse  
- [Jupyter Notebook](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-finder-tutorial-python/browse/GEDI_Finder_Tutorial_Python.ipynb)    

#### If you prefer to execute the code used in this tutorial outside of a Jupyter Notebook, a python script version is available:  
- [Python Script](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-finder-tutorial-python/browse/GEDI_Finder.py)  

---
# 1. Import Packages <a id="importpackages"></a>
#### All of the packages required to execute this tutorial are included in the Python Standard Library.

In [1]:
import requests as r
from datetime import datetime
import os

# 2. Define Function to Query CMR <a id="definefunction"></a>
#### In the code cell below, define a function called `gedi_finder` that takes two *user-submitted* input values, a `product` and a `bbox`.
#### There are three available products for this function, including 'GEDI01_B.002', 'GEDI02_A.002' and 'GEDI02_B.002'. 
#### A dictionary is set up to relate each product "shortname.version" to its associated "concept_id", which is a value used by NASA's CMR to retrieve data for a specific product. Additional information on concept ID's can be found in the [CMR Search API Documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-concept-id).

#### The second user-submitted input value, `bbox` is a string of bounding box coordinate values (decimal degrees) in the following format:
#### Lower Left Longitude, Lower Left Latitude, Upper Right Longitude, Upper Right Latitude ("LLLon,LLLat,URLon,URLat")  
> #### Example: `'-73.65,-12.64,-47.81,9.7'`

In [2]:
def gedi_finder(product, bbox):
    
    # Define the base CMR granule search url, including LPDAAC provider name and max page size (2000 is the max allowed)
    cmr = "https://cmr.earthdata.nasa.gov/search/granules.json?pretty=true&provider=LPDAAC_ECS&page_size=2000&concept_id="
    
    # Set up dictionary where key is GEDI shortname + version
    concept_ids = {'GEDI01_B.002': 'C1908344278-LPDAAC_ECS', 
                   'GEDI02_A.002': 'C1908348134-LPDAAC_ECS', 
                   'GEDI02_B.002': 'C1908350066-LPDAAC_ECS'}
    
    # CMR uses pagination for queries with more features returned than the page size
    page = 1
    bbox = bbox.replace(' ', '')  # remove any white spaces
    try:
        # Send GET request to CMR granule search endpoint w/ product concept ID, bbox & page number, format return as json
        cmr_response = r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']
        
        # If 2000 features are returned, move to the next page and submit another request, and append to the response
        while len(cmr_response) % 2000 == 0:
            page += 1
            cmr_response += r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']
        
        # CMR returns more info than just the Data Pool links, below use list comprehension to return a list of DP links
        return [c['links'][0]['href'] for c in cmr_response]
    except:
        # If the request did not complete successfully, print out the response from CMR
        print(r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox.replace(' ', '')}&pageNum={page}").json())

#### The function returns a list of links to download the intersecting GEDI sub-orbit V2 granules directly from the LP DAAC's Data Pool. 

# 3. Execute GEDI Finder Function <a id="executefunction"></a>
### Below is a demonstration of how to set the two required inputs to the `gedi_finder` function to variables.

In [3]:
# User-provided inputs (UPDATE FOR YOUR DESIRED PRODUCT AND BOUNDING BOX REGION OF INTEREST)
product = 'GEDI02_B.002'           # Options include 'GEDI01_B.002', 'GEDI02_A.002', 'GEDI02_B.002'
bbox = '-73.65,-12.64,-47.81,9.7'  # bounding box coordinates in LL Longitude, LL Latitude, UR Longitude, UR Latitude format

#### Above, the variables are defined to query the `GEDI02_B.002` product for a bounding box covering the Amazon Rainforest.

#### Next, call the `gedi_finder` function for the desired product and bounding box region of interest defined above, and set the output to a variable.

In [4]:
granules = gedi_finder(product, bbox)
print(f"{len(granules)} {product} Version 2 granules found.")

6558 GEDI02_B.002 Version 2 granules found.


#### Notice the print statement above will notify you how many granules intersected your bounding box for the product requested.

# 4. Export Results <a id="exportresults"></a>
#### Below is a demonstration of how to take the `granules` list of Data Pool links for intersecting GEDI V2 granules and export as a text file. The text file will be written to your current working directory, and will be named based on the date and time that the file was created. 

In [5]:
# Set up output text file name using the current datetime
outName = f"{product.replace('.', '_')}_GranuleList_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"

# Open file and write each granule link on a new line
with open(outName, "w") as gf:
    for g in granules:
        gf.write(f"{g}\n")
print(f"File containing links to intersecting {product} Version 2 data has been saved to:\n {os.getcwd()}\{outName}")

File containing links to intersecting GEDI02_B.002 Version 2 data has been saved to:
 c:\Users\mjami\Repository\GEDI-Data-Resources\Tutorials\GEDI02_B_002_GranuleList_20230329150801.txt


<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by LP DAAC^{1}$ </h3>
    <ul>
        <b>Contact:</b> LPDAAC@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6116 <br>
        <b>Organization:</b> Land Processes Distributed Active Archive Center (LP DAAC) <br>
        <b>Website:</b> https://lpdaac.usgs.gov/ <br>
        <b>Date last modified:</b> 05-21-2021 <br>
    </ul>
    
$^{1}$KBR Inc., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G15PD00467 for LP DAAC$^{2}$.

$^{2}$LP DAAC Work performed under NASA contract NNG14HH33I.
</div>